Code to install additional alibi_detect library in colab

In [ ]:
!pip install alibi_detect

### Imports

In [ ]:
from alibi_detect.models import PixelCNN
from alibi_detect.models.losses import elbo
from alibi_detect.models.autoencoder import eucl_cosim_features
from alibi_detect.od import OutlierVAE, OutlierVAEGMM, LLR
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.utils.fetching import fetch_detector
from alibi_detect.utils import visualize as viz
from alibi_detect.utils.perturbation import apply_mask
from alibi_detect.utils.prediction import predict_batch

import tensorflow as tf
from tensorflow.python.keras.engine.input_layer import InputLayer
from tensorflow.python.keras.layers import Conv2D, Dense, Reshape, Conv2DTranspose
from tensorflow.keras.metrics import MeanSquaredError, AUC, Precision, Recall

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm
import random

from google.colab import files

Check if GPU acceleration is enabled

In [ ]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Upload and unzip pretrained model files zip in colab

In [ ]:
files.upload()

In [ ]:
!unzip "detectors(3).zip"

Mount Drive to load models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/detectors"
plots_path = "/content/drive/MyDrive/plots"

Create folders for output

In [ ]:
try:
    os.mkdir("/content/detectors")
except FileExistsError as e:
    print("Directory exists already!")
try:
    os.mkdir("/content/plots")
except FileExistsError as e:
    print("Directory exists already!")
path = "/content/drive/MyDrive/detectors"
plots_path = "/content/plots"

# Cifar10

### Load the datasets

In [ ]:
dataset = 'cifar10'
train, test = tf.keras.datasets.cifar10.load_data()
X_train, y_train = train
X_test, y_test = test

# X_train = X_train[:15000]
# y_train = y_train[:15000]

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

X = X_train[:500]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X.shape)

input_shape=X_train.shape[1:]

In [ ]:
# randoms = [[random.randint(0, len(X_train)-1)] for _ in range(64)]
# plt.figure(figsize=(8,8), frameon=False)
# for i, idx in enumerate(randoms):
#     plt.subplot(8,8,i+1)
#     x = X_train[idx].reshape(1, 32, 32, 3)
#     plt.imshow(x.reshape(32, 32, 3))
#     plt.axis('off')
# plt.subplots_adjust(wspace=0,hspace=0)
# plt.savefig(os.path.join(plots_path, "cifar10"))
# plt.show()

## VAE


In [ ]:
fetch = False
load = True
epochs = 50
th = 0.1
s = 2
ld = 1024
bs = 56

In [ ]:
if fetch:
    detector_type = 'outlier'
    detector_name = 'OutlierVAE'
    dataset = 'cifar10'
    od_v = fetch_detector(path, detector_type, dataset, detector_name)
elif load:
    od_v = load_detector(os.path.join(path, 'OutlierVAE_Cifar10'))
else:
    encoder = tf.keras.Sequential([InputLayer(input_shape=input_shape),
                            Conv2D(32, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu)
                            ])
    
    decoder = tf.keras.Sequential([InputLayer(input_shape=(ld,)),
                            Dense(4*4*128),
                            Reshape(target_shape=(4, 4, 128)),
                            Conv2DTranspose(256, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2DTranspose(32, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2DTranspose(3, 4, strides=2, padding='same', activation='sigmoid')
                            ])

    od_v = OutlierVAE(threshold=None, 
                        score_type='mse', 
                        encoder_net=encoder,
                        decoder_net=decoder, 
                        latent_dim=ld, 
                        samples=s)


    od_v.fit(X_train, 
                loss_fn=elbo,
                cov_elbo=dict(sim=.05), 
                epochs=epochs, 
                batch_size=bs,
                verbose=True)

    od_v.infer_threshold(X, threshold_perc=99)  # assume 1% of the training data are outliers
    print('New threshold: {}'.format(od_v.threshold))

    save_detector(od_v, os.path.join(path, 'OutlierVAE_Cifar10'))


## VAEGMM

In [ ]:
load = True
epochs = 50
th = 0.1
s = 2 
ld = 4
n_gmm = 2
bs = 64

In [ ]:
if load:
    od_vg = load_detector(os.path.join(path, 'OutlierVAEGMM_Cifar10'))
else:
    encoder = tf.keras.Sequential([InputLayer(input_shape=input_shape),
                            Conv2D(5, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2D(10, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2D(20, 4, strides=2, padding='same', activation=tf.nn.relu)
                            ])
    
    decoder = tf.keras.Sequential([InputLayer(input_shape=(ld,)),
                            Dense(4*4*128),
                            Reshape(target_shape=(4, 4, 128)),
                            Conv2DTranspose(20, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2DTranspose(10, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2DTranspose(3, 4, strides=2, padding='same', activation='sigmoid')
                            ])

    gmm_density = tf.keras.Sequential([InputLayer(input_shape=(ld + 2,)),
                                Dense(10, activation=tf.nn.relu),
                                Dense(n_gmm, activation=tf.nn.softmax)
                                ])

    od_vg = OutlierVAEGMM(threshold=None, 
                            encoder_net=encoder, 
                            decoder_net=decoder, 
                            gmm_density_net=gmm_density, 
                            latent_dim=ld, 
                            n_gmm=n_gmm, 
                            samples=s, 
                            recon_features=eucl_cosim_features)


    losses = od_vg.fit(X_train, 
                        cov_elbo=dict(sim=.05),
                        epochs=epochs, 
                        batch_size=bs, 
                        verbose=True)
    
    print(losses)

    od_vg.infer_threshold(X, threshold_perc=99)  # assume 1% of the training data are outliers
    print('New threshold: {}'.format(od_vg.threshold))

    save_detector(od_vg, os.path.join(path, 'OutlierVAEGMM_Cifar10'))

## LR

In [ ]:
load = True

In [ ]:
model = PixelCNN(
    image_shape=input_shape,
    num_resnet=5,
    num_hierarchies=2,
    num_filters=32,
    num_logistic_mix=1,
    receptive_field_dims=(3, 3),
    dropout_p=.3,
    l2_weight=0.
)

if load:
    kwargs = {'dist_s': model, 'dist_b': model.copy(), 'input_shape': input_shape}
    od_lr = load_detector(os.path.join(path, "LR_Cifar10"), **kwargs)
else:
    od_lr = LLR(threshold=None, model=model)
    od_lr.fit(
            X_train,
            mutate_fn_kwargs=dict(rate=.2),
            mutate_batch_size=1000,
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            epochs=20,
            batch_size=32,
            verbose=True
        )
    
    od_lr.infer_threshold(X, threshold_perc=99)  # assume 1% of the training data are outliers
    print('New threshold: {}'.format(od_lr.threshold))

    save_detector(od_lr, os.path.join(path, 'LR_Cifar10'))

# MNIST/Fashion-MNIST

### Load the dataset

In [ ]:
def load_data(dataset: str) -> tuple:
    if dataset == 'mnist':
        (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    elif dataset == 'fashion_mnist':
        (X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    else:
        raise NotImplementedError
    X_train = X_train.astype('float32') / 255
    X_test = X_test.astype('float32') / 255
    y_train = y_train.astype('int64').reshape(-1,)
    y_test = y_test.astype('int64').reshape(-1,)
    if len(X_train.shape) == 3:
        shape = (-1,) + X_train.shape[1:] + (1,)
        X_train = X_train.reshape(shape)
        X_test = X_test.reshape(shape)
    return (X_train, y_train), (X_test, y_test)

(X_train_in, y_train_in), (X_test_in, y_test_in) = load_data('fashion_mnist')

X_train_in = np.pad(X_train_in,((0,0),(2,2),(2,2),(0,0)),constant_values=(0,))
X_test_in = np.pad(X_test_in,((0,0),(2,2),(2,2),(0,0)),constant_values=(0,))


X_test_ood, y_test_ood = load_data('mnist')[1]

X_test_ood = np.pad(X_test_ood, ((0,0),(2,2),(2,2),(0,0)), constant_values=(0,))


input_shape = X_train_in.shape[1:]

print(X_train_in.shape, X_test_in.shape, X_test_ood.shape)
print(y_train_in.shape, y_test_in.shape, y_test_ood.shape)

In [ ]:
randoms = [[random.randint(0, len(X_train_in)-1)] for _ in range(64)]
plt.figure(figsize=(8,8), frameon=False)
for i, idx in enumerate(randoms):
    plt.subplot(8,8,i+1)
    x = X_train_in[idx].reshape(1, 32, 32, 1)
    plt.imshow(x.reshape(32, 32))
    plt.axis('off')
plt.subplots_adjust(wspace=0,hspace=0)
plt.savefig(os.path.join(plots_path, "f_mnist"))
plt.show()

In [ ]:
randoms = [[random.randint(0, len(X_test_ood)-1)] for _ in range(64)]
plt.figure(figsize=(8,8), frameon=False)
for i, idx in enumerate(randoms):
    plt.subplot(8,8,i+1)
    x = X_test_ood[idx].reshape(1, 32, 32, 1)
    plt.imshow(x.reshape(32, 32))
    plt.axis('off')
plt.subplots_adjust(wspace=0,hspace=0)
plt.savefig(os.path.join(plots_path, "mnist"))
plt.show()

## VAE

In [ ]:
load = False
epochs = 50
th = 0.1
s = 2
ld = 1024
bs = 56

In [ ]:
if load:
    od_v1 = load_detector(os.path.join(path, 'OutlierVAE_FMNIST'))
else:
    encoder = tf.keras.Sequential([InputLayer(input_shape=input_shape),
                            Conv2D(32, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu)
                            ])
    
    decoder = tf.keras.Sequential([InputLayer(input_shape=(ld,)),
                            Dense(4*4*128),
                            Reshape(target_shape=(4, 4, 128)),
                            Conv2DTranspose(256, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2DTranspose(32, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2DTranspose(1, 4, strides=2, padding='same', activation='sigmoid')
                            ])


    od_v1 = OutlierVAE(threshold=None, 
                        score_type='mse', 
                        encoder_net=encoder,
                        decoder_net=decoder, 
                        latent_dim=ld, 
                        samples=s)

    od_v1.fit(X_train_in, 
                loss_fn=elbo,
                cov_elbo=dict(sim=.05), 
                epochs=epochs, 
                batch_size=bs,
                verbose=True)

    save_detector(od_v1, os.path.join(path, 'OutlierVAE_FMNIST'))


## VAEGMM

In [ ]:
load = False
epochs = 50
th = 0.1
s = 2 
ld = 4
n_gmm = 2
bs = 64

In [ ]:
if load:
    od_vg1 = load_detector(os.path.join(path, 'OutlierVAEGMM_FMNIST'))
else:
    encoder = tf.keras.Sequential([InputLayer(input_shape=input_shape),
                            Conv2D(5, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2D(10, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2D(20, 4, strides=2, padding='same', activation=tf.nn.relu)
                            ])
    
    decoder = tf.keras.Sequential([InputLayer(input_shape=(ld,)),
                            Dense(4*4*128),
                            Reshape(target_shape=(4, 4, 128)),
                            Conv2DTranspose(20, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2DTranspose(10, 4, strides=2, padding='same', activation=tf.nn.relu),
                            Conv2DTranspose(1, 4, strides=2, padding='same', activation='sigmoid')
                            ])

    gmm_density = tf.keras.Sequential([InputLayer(input_shape=(ld + 2,)),
                                Dense(10, activation=tf.nn.relu),
                                Dense(n_gmm, activation=tf.nn.softmax)
                                ])

    od_vg1 = OutlierVAEGMM(threshold=None, 
                            encoder_net=encoder, 
                            decoder_net=decoder, 
                            gmm_density_net=gmm_density, 
                            latent_dim=ld, 
                            n_gmm=n_gmm, 
                            samples=s, 
                            recon_features=eucl_cosim_features)


    od_vg1.fit(X_train_in, 
                cov_elbo=dict(sim=.05),
                epochs=epochs, 
                batch_size=bs, 
                verbose=True)

    save_detector(od_vg1, os.path.join(path, 'OutlierVAEGMM_FMNIST'))

## LR

In [ ]:
fetch = False
load = False

In [ ]:
model = PixelCNN(
    image_shape=input_shape,
    num_resnet=5,
    num_hierarchies=2,
    num_filters=32,
    num_logistic_mix=1,
    receptive_field_dims=(3, 3),
    dropout_p=.3,
    l2_weight=0.
)

if fetch:
    detector_type = 'outlier'
    dataset = 'fashion_mnist'
    detector_name = 'LLR'
    od_lr1 = fetch_detector(path, detector_type, dataset, detector_name)
elif load:
    kwargs = {'dist_s': model, 'dist_b': model.copy(), 'input_shape': input_shape}
    od_lr1 = load_detector(os.path.join(path, "LR_FMNIST"), **kwargs)
else:
    od_lr1 = LLR(threshold=None, model=model)
    od_lr1.fit(
            X_train_in,
            mutate_fn_kwargs=dict(rate=.2),
            mutate_batch_size=1000,
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            epochs=20,
            batch_size=32,
            verbose=True
        )
    save_detector(od_lr1, os.path.join(path, 'LR_FMNIST'))

## Infer thresholds

In [ ]:
shape_in, shape_ood = X_test_in.shape[0], X_test_ood.shape[0]

n, frac_outlier = 500, .5
perc_outlier = 100 * frac_outlier
n_in, n_ood = int(n * (1 - frac_outlier)), int(n * frac_outlier)
idx_in = np.random.choice(shape_in, size=n_in, replace=False)
idx_ood = np.random.choice(shape_ood, size=n_ood, replace=False)
X_threshold = np.concatenate([X_test_in[idx_in], X_test_ood[idx_ood]])

# od_v.infer_threshold(X_threshold, threshold_perc=perc_outlier, batch_size=56)
# od_vg.infer_threshold(X_threshold, threshold_perc=perc_outlier, batch_size=56)
od_lr1.infer_threshold(X_threshold, threshold_perc=perc_outlier, batch_size=32)
print(od_lr1.threshold)
print("done")

# Testing



## Cifar 10

Set thresholds

In [ ]:
od_v.infer_threshold(X, threshold_perc=99)  # assume 1% of the training data are outliers
od_vg.infer_threshold(X, threshold_perc=99)  # assume 1% of the training data are outliers
od_lr.infer_threshold(X, threshold_perc=99)  # assume 1% of the training data are outliers

In [ ]:
#VAE
od_v_preds = od_v.predict(X)

#VAEGMM
od_vg_preds = od_vg.predict(X)

#LR
od_lr_preds = od_lr.predict(X)

In [ ]:
from typing import Dict, Union
def plot_instance_score(preds: Dict,
                        target: np.ndarray,
                        labels: np.ndarray,
                        threshold: float,
                        ylim: tuple = (None, None)) -> None:

    scores = preds['data']['instance_score']
    df = pd.DataFrame(dict(idx=np.arange(len(scores)), score=scores, label=target))
    groups = df.groupby('label')
    fig, ax = plt.subplots()
    for name, group in groups:
        ax.plot(group.idx, group.score, marker='o', linestyle='', ms=6, label=labels[name])
    plt.plot(np.arange(len(scores)), np.ones(len(scores)) * threshold, color='g', label='Threshold')
    plt.ylim(ylim)
    plt.xlabel('Number of Instances')
    plt.ylabel('Instance Level Score')
    ax.legend()

# target = np.zeros(X.shape[0],).astype(int)  # all normal CIFAR10 training instances
# target = y_test.reshape([500])
# target = outlier_batch_target.reshape(outlier_batch_target.shape[0])
plt.figure()
labels = ['normal', 'outlier']

#VAE
target = od_v_preds['data']['is_outlier']
plot_instance_score(od_v_preds, target, labels, od_v.threshold)
plt.savefig(os.path.join(plots_path, "instance_score_vae.png"))
# plt.show()

#VAEGMM
target = od_vg_preds['data']['is_outlier']
plot_instance_score(od_vg_preds, target, labels, od_vg.threshold)
plt.savefig(os.path.join(plots_path, "instance_score_vaegmm.png"))
# plt.show()

#LR
target = od_lr_preds['data']['is_outlier']
plot_instance_score(od_lr_preds, target, labels, od_lr.threshold)
plt.savefig(os.path.join(plots_path, "instance_score_lr.png"))
# plt.show()

In [ ]:
n_mask_sizes = 10
n_masks = 20
n_imgs = 50

mask_sizes = [(2*n,2*n) for n in range(1,n_mask_sizes+1)]
print(mask_sizes)
img_ids = np.arange(n_imgs)
X_orig = X[img_ids].reshape(img_ids.shape[0], 32, 32, 3)
print(X_orig.shape)

In [ ]:
all_img_scores_v = []
all_img_scores_vg = []
all_img_scores_lr = []
for i in tqdm(range(X_orig.shape[0])):
    img_scores_v = np.zeros((len(mask_sizes),))
    img_scores_vg = np.zeros((len(mask_sizes),))
    img_scores_lr = np.zeros((len(mask_sizes),))
    for j, mask_size in enumerate(mask_sizes):
        # create masked instances
        X_mask, mask = apply_mask(X_orig[i].reshape(1, 32, 32, 3),
                                  mask_size=mask_size,
                                  n_masks=n_masks,
                                  channels=[0,1,2],
                                  mask_type='normal',
                                  noise_distr=(0,1),
                                  clip_rng=(0,1))
        # predict outliers
        od_v_preds_mask = od_v.predict(X_mask)
        od_vg_preds_mask = od_vg.predict(X_mask)
        od_lr_preds_mask = od_lr.predict(X_mask)

        score_v = od_v_preds_mask['data']['instance_score']
        score_vg = od_vg_preds_mask['data']['instance_score']
        score_lr = od_lr_preds_mask['data']['instance_score']

        # store average score over `n_masks` for a given mask size
        img_scores_v[j] = np.mean(score_v)
        img_scores_vg[j] = np.mean(score_vg)
        img_scores_lr[j] = np.mean(score_lr)

    all_img_scores_v.append(img_scores_v)
    all_img_scores_vg.append(img_scores_vg)
    all_img_scores_lr.append(img_scores_lr)

In [ ]:
x = X_train[34].reshape(1, 32, 32, 3)
x_mask, mask = apply_mask(x, mask_size=(12, 12), n_masks=1, channels=[0,1,2], 
                          mask_type='normal', noise_distr=(0,1), clip_rng=(0,1))
plt.imshow(x.reshape(32, 32, 3))
plt.axis('off')
plt.savefig(os.path.join(plots_path, "normal.png"))
plt.show()
plt.imshow(x_mask.reshape(32, 32, 3))
plt.axis('off')
plt.savefig(os.path.join(plots_path, "preturbed.png"))
plt.show()

In [ ]:
x_plt = [mask[0] for mask in mask_sizes]
for ais in all_img_scores_v:
    plt.plot(x_plt, ais)
    plt.xticks(x_plt)
plt.title('Outlier Score All Images for Increasing Mask Size')
plt.xlabel('Mask size')
plt.ylabel('Outlier Score')
plt.savefig(os.path.join(plots_path, "os_inc_mask_vae"))
plt.legend()
plt.show()

In [ ]:
for ais in all_img_scores_vg:
    plt.plot(x_plt, ais)
    plt.xticks(x_plt)
plt.title('Outlier Score All Images for Increasing Mask Size')
plt.xlabel('Mask size')
plt.ylabel('Outlier Score')
plt.savefig(os.path.join(plots_path, "os_inc_mask_vaegmm"))
plt.show()

In [ ]:
for ais in all_img_scores_lr:
    plt.plot(x_plt, ais)
    plt.xticks(x_plt)
plt.title('Outlier Score All Images for Increasing Mask Size')
plt.xlabel('Mask size')
plt.ylabel('Outlier Score')
plt.savefig(os.path.join(plots_path, "os_inc_mask_lr"))
plt.show()

In [ ]:
all_X_mask = []
X_i = X_orig[8].reshape(1, 32, 32, 3)
all_X_mask.append(X_i)
# apply masks
for j, mask_size in enumerate(mask_sizes):
    # create masked instances
    X_mask, mask = apply_mask(X_i,
                              mask_size=mask_size,
                              n_masks=1,  # just 1 for visualization purposes
                              channels=[0,1,2],
                              mask_type='normal',
                              noise_distr=(0,1),
                              clip_rng=(0,1))
    all_X_mask.append(X_mask)
all_X_mask = np.concatenate(all_X_mask, axis=0)
all_X_recon = od_v.vae(all_X_mask).numpy()
od_preds = od_v.predict(all_X_mask)

viz.plot_feature_outlier_image(od_v_preds,
                           all_X_mask,
                           X_recon=all_X_recon,
                           max_instances=all_X_mask.shape[0],
                           n_channels=3)

In [ ]:
X_recon = od_v.vae(X).numpy()
viz.plot_feature_outlier_image(od_v_preds,
                           X,
                           X_recon=X_recon,
                           instance_ids=[8, 15, 24, 39, 49])

## MNIST

In [ ]:
X_test = np.concatenate([X_test_in, X_test_ood])
y_test = np.concatenate([np.zeros(X_test_in.shape[0]), np.ones(X_test_ood.shape[0])])
print(X_test.shape, y_test.shape)

In [ ]:
#VAE
# od_v1_preds = od_v1.predict(X_test, batch_size=32)

# #VAEGMM
# od_vg1_preds = od_vg1.predict(X_test, batch_size=32)

#LR
od_lr1_preds = od_lr1.predict(X_test, batch_size=32)

In [ ]:
def metrics(od_preds):
    y_pred = od_preds['data']['is_outlier']
    labels = ['normal', 'outlier']
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    print('F1 score: {:.3f} -- Accuracy: {:.3f} -- Precision: {:.3f} '
        '-- Recall: {:.3f}'.format(f1, acc, prec, rec))
    cm = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(cm, index=labels, columns=labels)
    sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
    plt.show()

metrics(od_lr1_preds)

In [ ]:
def roc(od, od_preds):
    logp_s_in = predict_batch(od.dist_s.log_prob, X_test_in, batch_size=32, shape=shape_in)
    logp_s_ood = predict_batch(od.dist_s.log_prob, X_test_ood, batch_size=32, shape=shape_ood)
    logp_s = np.concatenate([logp_s_in, logp_s_ood])
    roc_data = {
        'LLR': {'scores': od_preds['data']['instance_score'], 'labels': y_test},
        'Likelihood': {'scores': -logp_s, 'labels': y_test}  # negative b/c outlier score
    }
    viz.plot_roc(roc_data)

roc(od_lr1, od_lr1_preds)